# Exploratory Data Analysis: ERCOT vs PJM Price Volatility

This notebook performs EDA on electricity market data from:
- **ERCOT West Zone** (Texas - energy-only market)
- **PJM DOM Zone** (Northern Virginia - capacity market)

## Objectives
1. Extract and clean data from both markets
2. Analyze price distributions and volatility patterns
3. Examine load patterns and their relationship to prices
4. Compare market characteristics

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# gridstatus for data extraction
import gridstatus

print(f"gridstatus version: {gridstatus.__version__}")
print(f"pandas version: {pd.__version__}")

## 1. Data Extraction

We'll extract 3 months of recent data for initial exploration.

In [ ]:
# Initialize market objects
ercot = gridstatus.Ercot()
pjm = gridstatus.PJM()

# Date range for initial exploration (3 months)
START_DATE = "Oct 1, 2024"
END_DATE = "Dec 31, 2024"

print(f"Extracting data from {START_DATE} to {END_DATE}")

### 1.1 ERCOT Data Extraction

In [ ]:
# Extract ERCOT LMP data (Settlement Point Prices)
print("Extracting ERCOT SPP data...")
ercot_lmp = ercot.get_lmp(
    start=START_DATE,
    end=END_DATE,
    market="REAL_TIME_15_MIN"
)
print(f"ERCOT LMP shape: {ercot_lmp.shape}")
ercot_lmp.head()

In [ ]:
# Extract ERCOT load data
print("Extracting ERCOT load data...")
ercot_load = ercot.get_load(
    start=START_DATE,
    end=END_DATE
)
print(f"ERCOT Load shape: {ercot_load.shape}")
ercot_load.head()

In [ ]:
# Extract ERCOT fuel mix
print("Extracting ERCOT fuel mix...")
ercot_fuel = ercot.get_fuel_mix(
    start=START_DATE,
    end=END_DATE
)
print(f"ERCOT Fuel Mix shape: {ercot_fuel.shape}")
ercot_fuel.head()

### 1.2 PJM Data Extraction

In [ ]:
# Extract PJM LMP data (zone level)
print("Extracting PJM LMP data...")
pjm_lmp = pjm.get_lmp(
    start=START_DATE,
    end=END_DATE,
    market="REAL_TIME_HOURLY",
    location_type="ZONE"
)
print(f"PJM LMP shape: {pjm_lmp.shape}")
pjm_lmp.head()

In [ ]:
# Extract PJM load data
print("Extracting PJM load data...")
pjm_load = pjm.get_load(
    start=START_DATE,
    end=END_DATE
)
print(f"PJM Load shape: {pjm_load.shape}")
pjm_load.head()

In [ ]:
# Extract PJM fuel mix
print("Extracting PJM fuel mix...")
pjm_fuel = pjm.get_fuel_mix(
    start=START_DATE,
    end=END_DATE
)
print(f"PJM Fuel Mix shape: {pjm_fuel.shape}")
pjm_fuel.head()

## 2. Data Exploration & Schema Understanding

In [ ]:
# Explore ERCOT LMP schema
print("ERCOT LMP Columns:")
print(ercot_lmp.columns.tolist())
print("\nERCOT LMP dtypes:")
print(ercot_lmp.dtypes)

In [ ]:
# Explore PJM LMP schema
print("PJM LMP Columns:")
print(pjm_lmp.columns.tolist())
print("\nPJM LMP dtypes:")
print(pjm_lmp.dtypes)

In [ ]:
# Check unique locations/zones
print("ERCOT unique locations:")
if 'Location' in ercot_lmp.columns:
    print(ercot_lmp['Location'].unique()[:20])
elif 'location' in ercot_lmp.columns:
    print(ercot_lmp['location'].unique()[:20])

print("\nPJM unique zones:")
if 'Location Name' in pjm_lmp.columns:
    print(pjm_lmp['Location Name'].unique()[:20])
elif 'location_name' in pjm_lmp.columns:
    print(pjm_lmp['location_name'].unique()[:20])

## 3. Data Cleaning & Transformation

In [ ]:
# Standardize column names
def standardize_columns(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

ercot_lmp = standardize_columns(ercot_lmp.copy())
ercot_load = standardize_columns(ercot_load.copy())
ercot_fuel = standardize_columns(ercot_fuel.copy())

pjm_lmp = standardize_columns(pjm_lmp.copy())
pjm_load = standardize_columns(pjm_load.copy())
pjm_fuel = standardize_columns(pjm_fuel.copy())

print("ERCOT LMP columns:", ercot_lmp.columns.tolist())
print("PJM LMP columns:", pjm_lmp.columns.tolist())

In [ ]:
# Filter ERCOT data for West zone (load zones: LZ_WEST or similar)
# Find the correct column for location
loc_col_ercot = [c for c in ercot_lmp.columns if 'location' in c.lower()]
print(f"ERCOT location columns: {loc_col_ercot}")

if len(loc_col_ercot) > 0:
    loc_col = loc_col_ercot[0]
    # Check what zones are available
    zones = ercot_lmp[loc_col].unique()
    west_zones = [z for z in zones if 'WEST' in str(z).upper() or 'LZ_WEST' in str(z).upper()]
    print(f"West-related zones found: {west_zones}")
    
    if len(west_zones) > 0:
        ercot_west = ercot_lmp[ercot_lmp[loc_col].isin(west_zones)].copy()
    else:
        # Use all hub/zone data
        hub_zones = [z for z in zones if 'HB_' in str(z) or 'LZ_' in str(z)]
        print(f"Using hub/load zones: {hub_zones[:5]}")
        ercot_west = ercot_lmp[ercot_lmp[loc_col].isin(hub_zones)].copy()
else:
    ercot_west = ercot_lmp.copy()

print(f"ERCOT West data shape: {ercot_west.shape}")

In [ ]:
# Filter PJM data for DOM zone
loc_col_pjm = [c for c in pjm_lmp.columns if 'location' in c.lower() or 'zone' in c.lower()]
print(f"PJM location columns: {loc_col_pjm}")

if len(loc_col_pjm) > 0:
    loc_col = loc_col_pjm[0]
    zones = pjm_lmp[loc_col].unique()
    dom_zones = [z for z in zones if 'DOM' in str(z).upper()]
    print(f"DOM zones found: {dom_zones}")
    
    if len(dom_zones) > 0:
        pjm_dom = pjm_lmp[pjm_lmp[loc_col].isin(dom_zones)].copy()
    else:
        pjm_dom = pjm_lmp.copy()
else:
    pjm_dom = pjm_lmp.copy()

print(f"PJM DOM data shape: {pjm_dom.shape}")

In [ ]:
# Find datetime and price columns
def find_datetime_col(df):
    for col in df.columns:
        if 'time' in col.lower() or 'date' in col.lower():
            if df[col].dtype == 'datetime64[ns]' or 'datetime' in str(df[col].dtype):
                return col
    return df.select_dtypes(include=['datetime64']).columns[0] if len(df.select_dtypes(include=['datetime64']).columns) > 0 else None

def find_price_col(df):
    for col in df.columns:
        if 'lmp' in col.lower() or 'price' in col.lower() or 'spp' in col.lower():
            if df[col].dtype in ['float64', 'int64']:
                return col
    return None

ercot_dt_col = find_datetime_col(ercot_west)
ercot_price_col = find_price_col(ercot_west)
pjm_dt_col = find_datetime_col(pjm_dom)
pjm_price_col = find_price_col(pjm_dom)

print(f"ERCOT: datetime={ercot_dt_col}, price={ercot_price_col}")
print(f"PJM: datetime={pjm_dt_col}, price={pjm_price_col}")

## 4. Price Distribution Analysis

In [ ]:
# Price statistics
if ercot_price_col and pjm_price_col:
    print("=" * 60)
    print("ERCOT West Price Statistics ($/MWh)")
    print("=" * 60)
    print(ercot_west[ercot_price_col].describe())
    
    print("\n" + "=" * 60)
    print("PJM DOM Price Statistics ($/MWh)")
    print("=" * 60)
    print(pjm_dom[pjm_price_col].describe())

In [ ]:
# Price distribution comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

if ercot_price_col:
    # ERCOT histogram
    ax1 = axes[0, 0]
    ercot_prices = ercot_west[ercot_price_col].dropna()
    # Clip for visualization
    ercot_prices_clipped = ercot_prices.clip(-50, 200)
    ax1.hist(ercot_prices_clipped, bins=100, alpha=0.7, color='steelblue', edgecolor='white')
    ax1.axvline(ercot_prices.median(), color='red', linestyle='--', label=f'Median: ${ercot_prices.median():.2f}')
    ax1.axvline(ercot_prices.mean(), color='orange', linestyle='--', label=f'Mean: ${ercot_prices.mean():.2f}')
    ax1.set_xlabel('Price ($/MWh)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('ERCOT West - Price Distribution')
    ax1.legend()

if pjm_price_col:
    # PJM histogram
    ax2 = axes[0, 1]
    pjm_prices = pjm_dom[pjm_price_col].dropna()
    pjm_prices_clipped = pjm_prices.clip(-50, 200)
    ax2.hist(pjm_prices_clipped, bins=100, alpha=0.7, color='forestgreen', edgecolor='white')
    ax2.axvline(pjm_prices.median(), color='red', linestyle='--', label=f'Median: ${pjm_prices.median():.2f}')
    ax2.axvline(pjm_prices.mean(), color='orange', linestyle='--', label=f'Mean: ${pjm_prices.mean():.2f}')
    ax2.set_xlabel('Price ($/MWh)')
    ax2.set_ylabel('Frequency')
    ax2.set_title('PJM DOM - Price Distribution')
    ax2.legend()

# Box plots
ax3 = axes[1, 0]
if ercot_price_col and pjm_price_col:
    data_for_box = [
        ercot_prices_clipped.values,
        pjm_prices_clipped.values
    ]
    bp = ax3.boxplot(data_for_box, labels=['ERCOT West', 'PJM DOM'], patch_artist=True)
    bp['boxes'][0].set_facecolor('steelblue')
    bp['boxes'][1].set_facecolor('forestgreen')
    ax3.set_ylabel('Price ($/MWh)')
    ax3.set_title('Price Distribution Comparison')

# Log-scale histogram for tail analysis
ax4 = axes[1, 1]
if ercot_price_col and pjm_price_col:
    ax4.hist(ercot_prices[ercot_prices > 0], bins=100, alpha=0.5, label='ERCOT West', color='steelblue')
    ax4.hist(pjm_prices[pjm_prices > 0], bins=100, alpha=0.5, label='PJM DOM', color='forestgreen')
    ax4.set_yscale('log')
    ax4.set_xlabel('Price ($/MWh)')
    ax4.set_ylabel('Frequency (log scale)')
    ax4.set_title('Price Distribution - Log Scale (Tail Analysis)')
    ax4.legend()

plt.tight_layout()
plt.savefig('../data/processed/price_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Volatility Analysis

In [ ]:
# Calculate hourly volatility metrics
def calculate_volatility_metrics(df, dt_col, price_col):
    """Calculate various volatility metrics."""
    df = df.copy()
    df = df.sort_values(dt_col)
    
    # Price returns
    df['returns'] = df[price_col].pct_change()
    df['log_returns'] = np.log(df[price_col] / df[price_col].shift(1))
    
    # Rolling volatility (standard deviation of returns)
    df['rolling_vol_24h'] = df['returns'].rolling(window=24, min_periods=12).std()
    df['rolling_vol_168h'] = df['returns'].rolling(window=168, min_periods=84).std()  # 7 days
    
    # Price range
    df['rolling_range_24h'] = df[price_col].rolling(window=24).max() - df[price_col].rolling(window=24).min()
    
    # Squared returns (proxy for variance)
    df['squared_returns'] = df['returns'] ** 2
    
    return df

if ercot_dt_col and ercot_price_col:
    ercot_vol = calculate_volatility_metrics(ercot_west, ercot_dt_col, ercot_price_col)
    print(f"ERCOT volatility data shape: {ercot_vol.shape}")

if pjm_dt_col and pjm_price_col:
    pjm_vol = calculate_volatility_metrics(pjm_dom, pjm_dt_col, pjm_price_col)
    print(f"PJM volatility data shape: {pjm_vol.shape}")

In [ ]:
# Volatility comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Returns distribution
ax1 = axes[0, 0]
if 'returns' in ercot_vol.columns:
    ercot_returns = ercot_vol['returns'].dropna()
    ercot_returns_clipped = ercot_returns.clip(-1, 1)
    ax1.hist(ercot_returns_clipped, bins=100, alpha=0.5, label='ERCOT West', color='steelblue', density=True)
if 'returns' in pjm_vol.columns:
    pjm_returns = pjm_vol['returns'].dropna()
    pjm_returns_clipped = pjm_returns.clip(-1, 1)
    ax1.hist(pjm_returns_clipped, bins=100, alpha=0.5, label='PJM DOM', color='forestgreen', density=True)
ax1.set_xlabel('Returns')
ax1.set_ylabel('Density')
ax1.set_title('Return Distribution Comparison')
ax1.legend()

# Rolling volatility time series
ax2 = axes[0, 1]
if 'rolling_vol_24h' in ercot_vol.columns and ercot_dt_col:
    ax2.plot(ercot_vol[ercot_dt_col], ercot_vol['rolling_vol_24h'], alpha=0.7, label='ERCOT West', color='steelblue')
if 'rolling_vol_24h' in pjm_vol.columns and pjm_dt_col:
    ax2.plot(pjm_vol[pjm_dt_col], pjm_vol['rolling_vol_24h'], alpha=0.7, label='PJM DOM', color='forestgreen')
ax2.set_xlabel('Date')
ax2.set_ylabel('24h Rolling Volatility')
ax2.set_title('Rolling Volatility Over Time')
ax2.legend()
ax2.tick_params(axis='x', rotation=45)

# Volatility distribution
ax3 = axes[1, 0]
if 'rolling_vol_24h' in ercot_vol.columns:
    ercot_vol_clean = ercot_vol['rolling_vol_24h'].dropna()
    ax3.hist(ercot_vol_clean.clip(0, 0.5), bins=50, alpha=0.5, label='ERCOT West', color='steelblue')
if 'rolling_vol_24h' in pjm_vol.columns:
    pjm_vol_clean = pjm_vol['rolling_vol_24h'].dropna()
    ax3.hist(pjm_vol_clean.clip(0, 0.5), bins=50, alpha=0.5, label='PJM DOM', color='forestgreen')
ax3.set_xlabel('24h Rolling Volatility')
ax3.set_ylabel('Frequency')
ax3.set_title('Volatility Distribution')
ax3.legend()

# Price range distribution
ax4 = axes[1, 1]
if 'rolling_range_24h' in ercot_vol.columns:
    ax4.hist(ercot_vol['rolling_range_24h'].dropna().clip(0, 200), bins=50, alpha=0.5, label='ERCOT West', color='steelblue')
if 'rolling_range_24h' in pjm_vol.columns:
    ax4.hist(pjm_vol['rolling_range_24h'].dropna().clip(0, 200), bins=50, alpha=0.5, label='PJM DOM', color='forestgreen')
ax4.set_xlabel('24h Price Range ($/MWh)')
ax4.set_ylabel('Frequency')
ax4.set_title('24-Hour Price Range Distribution')
ax4.legend()

plt.tight_layout()
plt.savefig('../data/processed/volatility_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Volatility statistics comparison
print("=" * 60)
print("Volatility Statistics Comparison")
print("=" * 60)

vol_stats = []

if 'returns' in ercot_vol.columns:
    ercot_stats = {
        'Market': 'ERCOT West',
        'Mean Return': ercot_vol['returns'].mean(),
        'Std Return': ercot_vol['returns'].std(),
        'Skewness': ercot_vol['returns'].skew(),
        'Kurtosis': ercot_vol['returns'].kurtosis(),
        'Mean 24h Vol': ercot_vol['rolling_vol_24h'].mean(),
        'Max 24h Vol': ercot_vol['rolling_vol_24h'].max(),
        'Mean 24h Range': ercot_vol['rolling_range_24h'].mean()
    }
    vol_stats.append(ercot_stats)

if 'returns' in pjm_vol.columns:
    pjm_stats = {
        'Market': 'PJM DOM',
        'Mean Return': pjm_vol['returns'].mean(),
        'Std Return': pjm_vol['returns'].std(),
        'Skewness': pjm_vol['returns'].skew(),
        'Kurtosis': pjm_vol['returns'].kurtosis(),
        'Mean 24h Vol': pjm_vol['rolling_vol_24h'].mean(),
        'Max 24h Vol': pjm_vol['rolling_vol_24h'].max(),
        'Mean 24h Range': pjm_vol['rolling_range_24h'].mean()
    }
    vol_stats.append(pjm_stats)

vol_stats_df = pd.DataFrame(vol_stats).set_index('Market').T
print(vol_stats_df.to_string())

## 6. Load Data Analysis

In [ ]:
# Explore load data structure
print("ERCOT Load columns:", ercot_load.columns.tolist())
print("\nERCOT Load sample:")
print(ercot_load.head())

In [ ]:
print("PJM Load columns:", pjm_load.columns.tolist())
print("\nPJM Load sample:")
print(pjm_load.head())

In [ ]:
# Load statistics
print("=" * 60)
print("ERCOT Load Statistics")
print("=" * 60)
print(ercot_load.describe())

print("\n" + "=" * 60)
print("PJM Load Statistics")
print("=" * 60)
print(pjm_load.describe())

## 7. Fuel Mix / Generation Analysis

In [ ]:
# Explore fuel mix structure
print("ERCOT Fuel Mix columns:", ercot_fuel.columns.tolist())
print("\nERCOT Fuel Mix sample:")
print(ercot_fuel.head())

In [ ]:
print("PJM Fuel Mix columns:", pjm_fuel.columns.tolist())
print("\nPJM Fuel Mix sample:")
print(pjm_fuel.head())

In [ ]:
# Calculate average fuel mix percentages
def calculate_fuel_percentages(df):
    """Calculate average generation by fuel type."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    fuel_cols = [c for c in numeric_cols if c not in ['hour', 'day', 'month', 'year']]
    
    if len(fuel_cols) > 0:
        avg_gen = df[fuel_cols].mean()
        total = avg_gen.sum()
        pct = (avg_gen / total * 100).sort_values(ascending=False)
        return pct
    return None

print("ERCOT Average Generation Mix:")
ercot_fuel_pct = calculate_fuel_percentages(ercot_fuel)
if ercot_fuel_pct is not None:
    print(ercot_fuel_pct.head(10))

print("\nPJM Average Generation Mix:")
pjm_fuel_pct = calculate_fuel_percentages(pjm_fuel)
if pjm_fuel_pct is not None:
    print(pjm_fuel_pct.head(10))

In [ ]:
# Fuel mix visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

if ercot_fuel_pct is not None:
    ax1 = axes[0]
    top_fuels = ercot_fuel_pct.head(6)
    colors = plt.cm.Set3(np.linspace(0, 1, len(top_fuels)))
    ax1.pie(top_fuels.values, labels=top_fuels.index, autopct='%1.1f%%', colors=colors)
    ax1.set_title('ERCOT Generation Mix')

if pjm_fuel_pct is not None:
    ax2 = axes[1]
    top_fuels = pjm_fuel_pct.head(6)
    colors = plt.cm.Set3(np.linspace(0, 1, len(top_fuels)))
    ax2.pie(top_fuels.values, labels=top_fuels.index, autopct='%1.1f%%', colors=colors)
    ax2.set_title('PJM Generation Mix')

plt.tight_layout()
plt.savefig('../data/processed/fuel_mix.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Temporal Patterns

In [ ]:
# Add temporal features to price data
def add_temporal_features(df, dt_col):
    df = df.copy()
    df['hour'] = df[dt_col].dt.hour
    df['day_of_week'] = df[dt_col].dt.dayofweek
    df['month'] = df[dt_col].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    return df

if ercot_dt_col:
    ercot_vol = add_temporal_features(ercot_vol, ercot_dt_col)
if pjm_dt_col:
    pjm_vol = add_temporal_features(pjm_vol, pjm_dt_col)

In [ ]:
# Hourly price patterns
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Average price by hour
ax1 = axes[0, 0]
if ercot_price_col and 'hour' in ercot_vol.columns:
    hourly_avg_ercot = ercot_vol.groupby('hour')[ercot_price_col].mean()
    ax1.plot(hourly_avg_ercot.index, hourly_avg_ercot.values, 'o-', label='ERCOT West', color='steelblue', linewidth=2)
if pjm_price_col and 'hour' in pjm_vol.columns:
    hourly_avg_pjm = pjm_vol.groupby('hour')[pjm_price_col].mean()
    ax1.plot(hourly_avg_pjm.index, hourly_avg_pjm.values, 's-', label='PJM DOM', color='forestgreen', linewidth=2)
ax1.set_xlabel('Hour of Day')
ax1.set_ylabel('Average Price ($/MWh)')
ax1.set_title('Average Price by Hour of Day')
ax1.legend()
ax1.set_xticks(range(0, 24, 2))

# Volatility by hour
ax2 = axes[0, 1]
if 'rolling_vol_24h' in ercot_vol.columns and 'hour' in ercot_vol.columns:
    hourly_vol_ercot = ercot_vol.groupby('hour')['rolling_vol_24h'].mean()
    ax2.plot(hourly_vol_ercot.index, hourly_vol_ercot.values, 'o-', label='ERCOT West', color='steelblue', linewidth=2)
if 'rolling_vol_24h' in pjm_vol.columns and 'hour' in pjm_vol.columns:
    hourly_vol_pjm = pjm_vol.groupby('hour')['rolling_vol_24h'].mean()
    ax2.plot(hourly_vol_pjm.index, hourly_vol_pjm.values, 's-', label='PJM DOM', color='forestgreen', linewidth=2)
ax2.set_xlabel('Hour of Day')
ax2.set_ylabel('Average 24h Rolling Volatility')
ax2.set_title('Volatility by Hour of Day')
ax2.legend()
ax2.set_xticks(range(0, 24, 2))

# Average price by day of week
ax3 = axes[1, 0]
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
if ercot_price_col and 'day_of_week' in ercot_vol.columns:
    daily_avg_ercot = ercot_vol.groupby('day_of_week')[ercot_price_col].mean()
    ax3.bar(np.array(daily_avg_ercot.index) - 0.2, daily_avg_ercot.values, width=0.4, label='ERCOT West', color='steelblue')
if pjm_price_col and 'day_of_week' in pjm_vol.columns:
    daily_avg_pjm = pjm_vol.groupby('day_of_week')[pjm_price_col].mean()
    ax3.bar(np.array(daily_avg_pjm.index) + 0.2, daily_avg_pjm.values, width=0.4, label='PJM DOM', color='forestgreen')
ax3.set_xlabel('Day of Week')
ax3.set_ylabel('Average Price ($/MWh)')
ax3.set_title('Average Price by Day of Week')
ax3.set_xticks(range(7))
ax3.set_xticklabels(day_names)
ax3.legend()

# Weekend vs Weekday comparison
ax4 = axes[1, 1]
comparison_data = []
if ercot_price_col and 'is_weekend' in ercot_vol.columns:
    ercot_weekday = ercot_vol[ercot_vol['is_weekend'] == 0][ercot_price_col].mean()
    ercot_weekend = ercot_vol[ercot_vol['is_weekend'] == 1][ercot_price_col].mean()
    comparison_data.append(['ERCOT Weekday', ercot_weekday])
    comparison_data.append(['ERCOT Weekend', ercot_weekend])
if pjm_price_col and 'is_weekend' in pjm_vol.columns:
    pjm_weekday = pjm_vol[pjm_vol['is_weekend'] == 0][pjm_price_col].mean()
    pjm_weekend = pjm_vol[pjm_vol['is_weekend'] == 1][pjm_price_col].mean()
    comparison_data.append(['PJM Weekday', pjm_weekday])
    comparison_data.append(['PJM Weekend', pjm_weekend])

if comparison_data:
    comp_df = pd.DataFrame(comparison_data, columns=['Category', 'Avg Price'])
    colors = ['steelblue', 'lightsteelblue', 'forestgreen', 'lightgreen']
    ax4.bar(comp_df['Category'], comp_df['Avg Price'], color=colors[:len(comp_df)])
    ax4.set_ylabel('Average Price ($/MWh)')
    ax4.set_title('Weekday vs Weekend Price Comparison')
    ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../data/processed/temporal_patterns.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Price Spikes Analysis

In [ ]:
# Identify price spikes
def identify_spikes(df, price_col, threshold_multiplier=3):
    """Identify price spikes as values > threshold_multiplier * rolling mean."""
    df = df.copy()
    rolling_mean = df[price_col].rolling(window=168, min_periods=24).mean()  # 7-day rolling mean
    df['spike_threshold'] = rolling_mean * threshold_multiplier
    df['is_spike'] = (df[price_col] > df['spike_threshold']).astype(int)
    
    # Also flag extreme spikes (> 5x)
    df['is_extreme_spike'] = (df[price_col] > rolling_mean * 5).astype(int)
    
    return df

if ercot_price_col:
    ercot_vol = identify_spikes(ercot_vol, ercot_price_col)
    print(f"ERCOT Spikes (>3x): {ercot_vol['is_spike'].sum()} ({ercot_vol['is_spike'].mean()*100:.2f}%)")
    print(f"ERCOT Extreme Spikes (>5x): {ercot_vol['is_extreme_spike'].sum()} ({ercot_vol['is_extreme_spike'].mean()*100:.2f}%)")

if pjm_price_col:
    pjm_vol = identify_spikes(pjm_vol, pjm_price_col)
    print(f"\nPJM Spikes (>3x): {pjm_vol['is_spike'].sum()} ({pjm_vol['is_spike'].mean()*100:.2f}%)")
    print(f"PJM Extreme Spikes (>5x): {pjm_vol['is_extreme_spike'].sum()} ({pjm_vol['is_extreme_spike'].mean()*100:.2f}%)")

In [ ]:
# Spike distribution by hour
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if 'is_spike' in ercot_vol.columns and 'hour' in ercot_vol.columns:
    spike_by_hour_ercot = ercot_vol.groupby('hour')['is_spike'].mean() * 100
    axes[0].bar(spike_by_hour_ercot.index, spike_by_hour_ercot.values, color='steelblue', alpha=0.7)
    axes[0].set_xlabel('Hour of Day')
    axes[0].set_ylabel('Spike Probability (%)')
    axes[0].set_title('ERCOT West - Price Spike Probability by Hour')
    axes[0].set_xticks(range(0, 24, 2))

if 'is_spike' in pjm_vol.columns and 'hour' in pjm_vol.columns:
    spike_by_hour_pjm = pjm_vol.groupby('hour')['is_spike'].mean() * 100
    axes[1].bar(spike_by_hour_pjm.index, spike_by_hour_pjm.values, color='forestgreen', alpha=0.7)
    axes[1].set_xlabel('Hour of Day')
    axes[1].set_ylabel('Spike Probability (%)')
    axes[1].set_title('PJM DOM - Price Spike Probability by Hour')
    axes[1].set_xticks(range(0, 24, 2))

plt.tight_layout()
plt.savefig('../data/processed/spike_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Summary Statistics & Key Findings

In [ ]:
# Create comprehensive summary
summary = {
    'Metric': [],
    'ERCOT West': [],
    'PJM DOM': []
}

# Price metrics
if ercot_price_col and pjm_price_col:
    metrics = [
        ('Mean Price ($/MWh)', ercot_vol[ercot_price_col].mean(), pjm_vol[pjm_price_col].mean()),
        ('Median Price ($/MWh)', ercot_vol[ercot_price_col].median(), pjm_vol[pjm_price_col].median()),
        ('Std Dev Price', ercot_vol[ercot_price_col].std(), pjm_vol[pjm_price_col].std()),
        ('Max Price ($/MWh)', ercot_vol[ercot_price_col].max(), pjm_vol[pjm_price_col].max()),
        ('Min Price ($/MWh)', ercot_vol[ercot_price_col].min(), pjm_vol[pjm_price_col].min()),
        ('Price Skewness', ercot_vol[ercot_price_col].skew(), pjm_vol[pjm_price_col].skew()),
        ('Price Kurtosis', ercot_vol[ercot_price_col].kurtosis(), pjm_vol[pjm_price_col].kurtosis()),
    ]
    
    # Volatility metrics
    if 'rolling_vol_24h' in ercot_vol.columns and 'rolling_vol_24h' in pjm_vol.columns:
        metrics.extend([
            ('Mean 24h Volatility', ercot_vol['rolling_vol_24h'].mean(), pjm_vol['rolling_vol_24h'].mean()),
            ('Max 24h Volatility', ercot_vol['rolling_vol_24h'].max(), pjm_vol['rolling_vol_24h'].max()),
        ])
    
    # Spike metrics
    if 'is_spike' in ercot_vol.columns and 'is_spike' in pjm_vol.columns:
        metrics.extend([
            ('Spike Rate (>3x, %)', ercot_vol['is_spike'].mean() * 100, pjm_vol['is_spike'].mean() * 100),
            ('Extreme Spike Rate (>5x, %)', ercot_vol['is_extreme_spike'].mean() * 100, pjm_vol['is_extreme_spike'].mean() * 100),
        ])
    
    for metric_name, ercot_val, pjm_val in metrics:
        summary['Metric'].append(metric_name)
        summary['ERCOT West'].append(f"{ercot_val:.4f}" if isinstance(ercot_val, float) else ercot_val)
        summary['PJM DOM'].append(f"{pjm_val:.4f}" if isinstance(pjm_val, float) else pjm_val)

summary_df = pd.DataFrame(summary)
print("\n" + "=" * 70)
print("SUMMARY: ERCOT West vs PJM DOM Price Volatility Comparison")
print("=" * 70)
print(summary_df.to_string(index=False))

In [ ]:
# Save processed data
print("\nSaving processed data...")

# Save to parquet
if len(ercot_vol) > 0:
    ercot_vol.to_parquet('../data/processed/ercot_west_processed.parquet', index=False)
    print(f"Saved ERCOT data: {len(ercot_vol)} rows")

if len(pjm_vol) > 0:
    pjm_vol.to_parquet('../data/processed/pjm_dom_processed.parquet', index=False)
    print(f"Saved PJM data: {len(pjm_vol)} rows")

# Save summary
summary_df.to_csv('../data/processed/eda_summary.csv', index=False)
print("Saved summary statistics")

## 11. Key Findings

### Price Distribution
- Compare mean, median, and standard deviation between markets
- Note any differences in skewness and kurtosis (tail behavior)

### Volatility Patterns
- Compare rolling volatility levels
- Identify peak volatility hours
- Note weekend vs weekday differences

### Price Spikes
- Compare spike frequency between markets
- Identify hours most prone to spikes
- Note any seasonal patterns

### Market Structure Implications
- ERCOT (energy-only market) vs PJM (capacity market) differences
- Impact of renewable penetration
- Implications for data center operations

In [ ]:
print("\n" + "=" * 70)
print("EDA COMPLETE")
print("=" * 70)
print("\nGenerated files:")
print("  - data/processed/ercot_west_processed.parquet")
print("  - data/processed/pjm_dom_processed.parquet")
print("  - data/processed/eda_summary.csv")
print("  - data/processed/price_distribution.png")
print("  - data/processed/volatility_analysis.png")
print("  - data/processed/fuel_mix.png")
print("  - data/processed/temporal_patterns.png")
print("  - data/processed/spike_analysis.png")